In [30]:
import os
import csv
import pickle
import re

def renamefile():
    location = os.getcwd()
    path = os.path.join(location, 'static/iconset/svg')
    for filename in os.listdir(path):
        os.rename(os.path.join(path,filename), os.path.join(path,filename[4:]))

def file2csv():
    location = os.getcwd()
    path = location + '/static/iconset'
    with open('iconbase.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for filename in os.listdir(path):
            writer.writerow([filename])
            
def csv2list(csvname):
    with open(csvname, mode='r') as csvfile:
        reader = csv.reader(csvfile)
        filelist = list(reader)
        return filelist

def list2keywords(filelist):
    keywordslist = []
    for filename in filelist:
        name = filename[0].split(".")[0] # delete .scv
        keywords = re.split('[-_\s]',name)
        for keyword in keywords:
            if keyword.isdigit():
                keywords.remove(keyword)
        row = filename + keywords
        keywordslist.append(row)
    return keywordslist

def listtocsv(keywordslist,csvname):
    with open(csvname, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for x in keywordslist:
            writer.writerow(x)
            
# turn icon files to keyword list csv           
def csv2keywordscsv(src, tgt):
    namelist = csv2list(src)
    keywordslist=list2keywords(namelist)
    listtocsv(keywordslist, tgt)
                  
    
def csv2dict():
    icondict= {}
    with open('iconset.csv', mode='r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            wordlist = []
            for i in range(1, len(row)):
                if row[i]:
                    wordlist.append(row[i].lower())
            icondict[row[0]] = wordlist
    return icondict

#input = {iconname: [word1, word2, ...], ...}
#res = {word: [iconname1, iconname2, ...], ...}
def fullIndex(icondict):
    total_index = {}
    for iconname in icondict.keys():
        for word in icondict[iconname]:
            if word in total_index.keys():
                total_index[word].append(iconname)
            else:
                total_index[word] = [iconname]
            
    return total_index

def save_dict(obj, name):
    if not os.path.exists('/data'):
        os.makedirs('/data')
    with open('data/'+name+'.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_dict(name):
    with open('data/'+name+'.pkl', 'rb') as f:
        return pickle.load(f)
        


In [18]:
file2csv()

In [31]:
csv2keywordscsv('iconbase.csv', 'iconset.csv')

In [32]:
index = fullIndex(csv2dict())
save_dict(index, 'icondata')
print(len(load_dict('icondata')))

947


In [ ]:
!python server.py